In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import time
from datetime import datetime
from google.cloud.logging import DESCENDING, Client
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['axes.titlesize'] = 18

In [2]:
client = Client(project='ultra-sound-relay')

/usr/local/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [17]:
now = pd.Timestamp.now()
pdFilter = 'jsonPayload.method="getPayload" AND "execution payload delivered" ' + \
           'AND timestamp>=' + '\"' + (now - pd.Timedelta('5h')).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\" ' + \
           'AND timestamp<=' + '\"' + (now).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\"'

In [18]:
payloads = []
for e in client.list_entries(filter_=pdFilter):
    payloads.append(e.to_api_repr()['jsonPayload'])
payloads = pd.DataFrame(payloads)

In [19]:
payloads.head()

,timestampRequestStart,proposerPubkey,timestampAfterDecode,timestampAfterAlreadyDelivered,method,ua,mevBoostV,slot,slotStartSec,timestampAfterSignatureVerify,...,contentLength,version,msIntoSlot,timestampAfterPublishing,idArg,service,level,msg,numTx,blockNumber
0,1.681358e+12,0x84ee9fab8a3a27354d8115d354b1846518c8133499f8...,1.681358e+12,1.681358e+12,getPayload,mev-boost/v1.5.0,v1.5.0,6211161.0,1.681358e+09,1.681358e+12,...,57892.0,,620.0,1.681358e+12,,relay/api,info,execution payload delivered,119.0,17036256.0
1,1.681358e+12,0x93b77a2c697a113ffa07e68e16cc6b4f1da63fabe1c1...,1.681358e+12,1.681358e+12,getPayload,mev-boost/v1.5.0 okhttp/4.10.0,v1.5.0,6211167.0,1.681358e+09,1.681358e+12,...,91899.0,,713.0,1.681358e+12,,relay/api,info,execution payload delivered,146.0,17036261.0
2,1.681358e+12,0x9563088f6a10ea94eef35511b2da855c8f635bde5811...,1.681358e+12,1.681358e+12,getPayload,mev-boost/v1.5.0,v1.5.0,6211170.0,1.681358e+09,1.681358e+12,...,55319.0,,1068.0,1.681358e+12,,relay/api,info,execution payload delivered,141.0,17036264.0
3,1.681358e+12,0x99520e45bfd21939316d33f48ec8b657ee8ca7d0225b...,1.681358e+12,1.681358e+12,getPayload,mev-boost/v1.5.0 nim-presto/0.0.3 (amd64/linux),v1.5.0,6211174.0,1.681358e+09,1.681358e+12,...,91859.0,,771.0,1.681358e+12,,relay/api,info,execution payload delivered,293.0,17036268.0
4,1.681358e+12,0x86539cf7a4b1cda4780f208e7c24b2d62993b9889702...,1.681358e+12,1.681358e+12,getPayload,mev-boost/v1.5.0 okhttp/4.10.0,v1.5.0,6211180.0,1.681358e+09,1.681358e+12,...,91873.0,,601.0,1.681358e+12,btcs,relay/api,info,execution payload delivered,121.0,17036274.0


In [20]:
payloads['ua'].value_counts()

mev-boost/v1.5.0                                                           125
mev-boost/v1.5.0 okhttp/4.10.0                                              67
Go-http-client/1.1                                                          66
mev-boost/v1.5.0 Prysm/v4.0.2/75338dd83d943ed5e13635cd0578c38a3b2d248b      15
mev-boost/v1.5.0 nim-presto/0.0.3 (amd64/linux)                              8
mev-boost/v1.5.0 node-fetch/1.0 (+https://github.com/bitinn/node-fetch)      5
mev-boost/v1.5.1-alpha1                                                      2
                                                                             1
Name: ua, dtype: int64